In [59]:
%matplotlib inline
import numpy as np
import pandas as pd
pd.options.display.max_rows = 10
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor

In [2]:
unusuals_df = pd.read_csv("CompleteHatAndPricingData.csv")
unusuals_df = unusuals_df[unusuals_df.columns.drop("Unnamed: 0")]
unusuals_df

,Name,Effect,Price,Class,Item Type,Equip Region,Grade,Unboxed From,Robo,Generation,Effect Community Rating,Crate Cosmetic,Months Since Last Price Update,Effect Median Price,Hat Median Price,Hat Rating,Effect Rating,Median Effect Rating,Median Hat Rating
0,Dread Knot,Burning Flames,30.0,Heavy,Cosmetic,Regular,No Grade,Old Series,False,First Gen,4.59375,True,9,104.00,14.0,-7.40,1.6,6.7375,-1.6000
1,Dread Knot,Circling Heart,NaN,Heavy,Cosmetic,Regular,No Grade,Old Series,False,First Gen,3.06250,True,-1,36.25,14.0,NaN,NaN,0.4500,-1.6000
2,Dread Knot,Circling Peace Sign,10.0,Heavy,Cosmetic,Regular,No Grade,Old Series,False,First Gen,2.21875,True,21,22.00,14.0,-1.20,-0.4,-0.8500,-1.6000
3,Dread Knot,Circling TF Logo,9.0,Heavy,Cosmetic,Regular,No Grade,Old Series,False,First Gen,2.34375,True,44,21.50,14.0,-1.25,-0.5,-0.9125,-1.6000
4,Dread Knot,Green Confetti,9.0,Heavy,Cosmetic,Regular,No Grade,Old Series,False,First Gen,1.40625,True,15,17.00,14.0,-0.80,-0.5,-1.3750,-1.6000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41276,Snowcapped,Magnetic Hat Protector,NaN,Medic,Cosmetic,Regular,Mercenary,Winter 2018 Cosmetic,False,Invasion,2.15625,False,-1,20.50,22.0,NaN,NaN,-0.9000,-0.1625
41277,Snowcapped,Voltaic Hat Protector,NaN,Medic,Cosmetic,Regular,Mercenary,Winter 2018 Cosmetic,False,Invasion,2.09375,False,-1,52.50,22.0,NaN,NaN,-1.1750,-0.1625
41278,Snowcapped,Galactic Codex,NaN,Medic,Cosmetic,Regular,Mercenary,Winter 2018 Cosmetic,False,Invasion,1.43750,False,-1,36.75,22.0,NaN,NaN,-1.1000,-0.1625
41279,Snowcapped,Ancient Codex,NaN,Medic,Cosmetic,Regular,Mercenary,Winter 2018 Cosmetic,False,Invasion,1.46875,False,-1,19.50,22.0,NaN,NaN,-1.2500,-0.1625


In [3]:
unusuals_train_df = unusuals_df[~unusuals_df["Price"].isnull()]
unusuals_predict_df = unusuals_df[unusuals_df["Price"].isnull()]

In [60]:
cross_val_subset = unusuals_train_df[(unusuals_train_df["Equip Region"] == "Regular") & (unusuals_train_df["Price"] <= 2000)]

In [61]:
feature_set = cross_val_subset.columns.drop(["Price","Effect","Unboxed From","Months Since Last Price Update",
                                             "Effect Median Price","Hat Median Price","Hat Rating","Effect Rating"])
feature_set

Index(['Name', 'Class', 'Item Type', 'Equip Region', 'Grade', 'Robo',
       'Generation', 'Effect Community Rating', 'Crate Cosmetic',
       'Median Effect Rating', 'Median Hat Rating'],
      dtype='object')

In [6]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.linear_model import LinearRegression

class RegressionEnsembler(BaseEstimator, RegressorMixin):
    """Creates an ensemble model out of a collection of individual estimators
    
    Args:
      estimators: A list containing the individual estimators.
      learn_weights: A boolean that specifies whether we should learn the
        "optimal" weights/coefficients to apply to each individual estimator's
        predictions. If False, we simply return the straight average of the 
        individual estimators' predictions as the ensemble prediction.
    """
    
    def __init__(self, estimators, learn_weights=True):
        self.estimators = estimators
        self.learn_weights = learn_weights
        self.vec = DictVectorizer(sparse=False)
        self.scaler = StandardScaler()
        
    def fit(self, X, y):
        
        # store the training features and the labels
        self.X_ = X
        self.y_ = y
        
        # call the fit method of each of the estimators
        for estimator in self.estimators:
            #X_train_dict = X.to_dict(orient = "records")
            
            estimator.fit(X, y) #X_train_sc
            
        # if we wish to learn the "optimal" weights from the training data
        if self.learn_weights:
            # get prediction from each estimator on the training data
            predictions = []
            for estimator in self.estimators:
                predictions.append(estimator.predict(X))
            Y_ = np.column_stack(predictions)
        
            # fit linear regression on top of the estimators' predictions
            self.ensembler = LinearRegression(fit_intercept=False)
            self.ensembler.fit(Y_, y)
            
        return self
    
    def predict(self, X):
        # check that fit has been called
        check_is_fitted(self, ['X_', 'y_'])
        
        # calculate predictions from the estimators
        predictions = []
        for estimator in self.estimators:
            
            #print(estimator.predict(X_pred_sc))
            predictions.append(estimator.predict(X))
        Y_ = np.column_stack(predictions)
        
        # return predictions
        if self.learn_weights:
            return self.ensembler.predict(Y_)
        else:
            return Y_.mean(axis=1)

In [7]:
def get_error_from_feature_set(feature_set,df):
    X_train = df[feature_set]
    X_dict = df[feature_set].to_dict(orient = "records")
    y_train = df["Price"]
    
    vec = DictVectorizer(sparse=False)
    scaler = StandardScaler()
    model1 = KNeighborsRegressor(n_neighbors=15,metric = "manhattan")
    model2 = RandomForestRegressor(n_estimators=50)    
    model3 = BaggingRegressor()
    
    
    model_1_pipeline = Pipeline([("vectorizer",vec),("scaler",scaler),("fit",model1)])
    model_2_pipeline = Pipeline([("vectorizer",vec),("scaler",scaler),("fit",model2)])
    model_3_pipeline = Pipeline([("vectorizer",vec),("scaler",scaler),("fit",model3)])
    
    scores_1 = cross_val_score(model_1_pipeline, X_dict, y_train,
                               cv=3, scoring="neg_mean_squared_error")
    scores_2 = cross_val_score(model_2_pipeline, X_dict, y_train,
                               cv=3, scoring="neg_mean_squared_error")
    
    scores_3 = cross_val_score(model_3_pipeline, X_dict, y_train,
                               cv=3, scoring="neg_mean_squared_error")
        
    ensemble_model_1 = RegressionEnsembler([model_1_pipeline, model_2_pipeline,model_3_pipeline],learn_weights=False)
    ensemble_model_2 = RegressionEnsembler([model_1_pipeline, model_2_pipeline,model_3_pipeline],learn_weights=True)
    
    scores_4 = cross_val_score(ensemble_model_1, X_dict, y_train, cv=3, scoring="neg_mean_squared_error")
    scores_5 = cross_val_score(ensemble_model_2, X_dict, y_train, cv=3, scoring="neg_mean_squared_error")
    
    print(scores_4)
    print(scores_5)
    
    return [np.sqrt(np.mean(-scores_1)),np.sqrt(np.mean(-scores_2)),np.sqrt(np.mean(-scores_3)),np.sqrt(np.mean(-scores_4)),np.sqrt(np.mean(-scores_5))]

In [8]:
def predict_set(feature_set,train_df,predict_df):
    
    X_dict = train_df[feature_set].to_dict(orient = "records")
    X_validation_dict = predict_df[feature_set].to_dict(orient = "records")
    y = train_df["Price"]
    y_validation = predict_df["Price"]
    
    vec = DictVectorizer(sparse=False)
    vec.fit(X_dict)
    X_train = vec.transform(X_dict)
    X_validation = vec.transform(X_validation_dict)
    
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_validation_scaled = scaler.transform(X_validation)
        
    model_1 = KNeighborsRegressor(n_neighbors=5,metric = "manhattan")
    model_1.fit(X_train_scaled,y)
    y_validation_pred_model_1 = model_1.predict(X_validation_scaled)
    
    model_2 = RandomForestRegressor(n_estimators=50) 
    model_2.fit(X_train_scaled,y)
    y_validation_pred_model_2 = model_2.predict(X_validation_scaled)
    
    model_3 = BaggingRegressor()
    model_3.fit(X_train_scaled,y)
    y_validation_pred_model_3 = model_3.predict(X_validation_scaled)
    
    model_1_pipeline = Pipeline([("vectorizer",vec),("scaler",scaler),("fit",model_1)])
    model_2_pipeline = Pipeline([("vectorizer",vec),("scaler",scaler),("fit",model_2)])
    model_3_pipeline = Pipeline([("vectorizer",vec),("scaler",scaler),("fit",model_3)])
    
    ensemble_model_1 = RegressionEnsembler([model_1_pipeline, model_2_pipeline,model_3_pipeline],learn_weights=False)
    ensemble_model_2 = RegressionEnsembler([model_1_pipeline, model_2_pipeline,model_3_pipeline],learn_weights=True)
    
    ensemble_model_1.fit(X_dict,y)
    y_validation_pred_model_4 = ensemble_model_1.predict(X_validation_dict)
    
    ensemble_model_2.fit(X_dict,y)
    y_validation_pred_model_5 = ensemble_model_2.predict(X_validation_dict)
    
    return [y_validation_pred_model_1, y_validation_pred_model_2,y_validation_pred_model_3,y_validation_pred_model_4,y_validation_pred_model_5]

In [47]:
get_error_from_feature_set(feature_set,cross_val_subset)

[-1622.26164825 -2530.64849444 -3781.10914689]
[-1880.12424841 -2777.03996366 -3506.63125424]


[69.93669959453935,
 50.12324616688101,
 51.6225419603154,
 51.42638521736162,
 52.165746955621756]

In [65]:
pd.Series([1,2,3]).median()

2.0

In [129]:
result = get_error_from_feature_set(feature_set,cross_val_subset)

[-5223.49509775 -6134.3094885  -8296.40617706]
[-5454.27207098 -6551.34592935 -9244.96385709]


In [130]:
result

[84.121286332013241,
 83.950228916168172,
 83.49272319751644,
 80.940741210891531,
 84.163693394527826]

In [48]:
prediction_results = predict_set(feature_set,unusuals_train_df,unusuals_predict_df)
prediction_results

[array([10.9 ,  9.2 ,  8.45, ..., 20.4 , 20.3 , 51.8 ]),
 array([ 16.81,   9.46,   8.7 , ...,  21.06,  15.98, 115.92]),
 array([ 14.7  ,   9.9  ,   9.025, ...,  21.6  ,  15.2  , 102.85 ]),
 array([12.9       ,  9.35      ,  8.60333333, ..., 20.34666667,
        16.72      , 94.42666667]),
 array([ 17.40034744,   9.66638596,   8.93839808, ...,  20.24509054,
         15.91571112, 119.51348036])]

In [49]:
pd.DataFrame(np.array(prediction_results).T)

,0,1,2,3,4
0,10.90,16.81,14.700,12.900000,17.400347
1,9.20,9.46,9.900,9.350000,9.666386
2,8.45,8.70,9.025,8.603333,8.938398
3,32.40,58.83,58.850,48.456667,61.863571
4,32.40,54.36,53.900,47.200000,54.210949
...,...,...,...,...,...
24688,21.00,19.49,19.150,20.583333,18.920388
24689,20.80,17.87,17.250,18.706667,17.861942
24690,20.40,21.06,21.600,20.346667,20.245091
24691,20.30,15.98,15.200,16.720000,15.915711


In [50]:
unusuals_final_predictions_df = (pd.concat([unusuals_predict_df.reset_index(),pd.DataFrame(np.array(prediction_results).T)],axis = 1).
                                rename(columns = {0:"Predicted Price KNN",1:"Predicted Price Forest",2:"Bagging",3:"Learn Weights = False",4:"Learn Weights = True"}))
unusuals_final_predictions_df = unusuals_final_predictions_df[unusuals_final_predictions_df.columns.drop("Price")]

In [51]:
unusuals_final_predictions_df["Price Pred. Average"] = (unusuals_final_predictions_df["Predicted Price KNN"] + 
                                                        unusuals_final_predictions_df["Predicted Price Forest"] + 
                                                        unusuals_final_predictions_df["Learn Weights = True"] + 
                                                       unusuals_final_predictions_df["Learn Weights = False"])/4
unusuals_final_predictions_df_short = unusuals_final_predictions_df[["Name","Effect","Predicted Price KNN", "Predicted Price Forest","Bagging","Learn Weights = False",
                                                                                     "Learn Weights = True","Price Pred. Average"]]

In [57]:
print(unusuals_final_predictions_df_short[unusuals_final_predictions_df_short["Name"] == "Backwards Ballcap"].sort_values("Price Pred. Average").to_string())

                   Name                            Effect  Predicted Price KNN  Predicted Price Forest  Bagging  Learn Weights = False  Learn Weights = True  Price Pred. Average
5567  Backwards Ballcap            Electric Hat Protector                 29.2                   16.82    20.50              21.660000             17.850510            21.382627
5571  Backwards Ballcap                     Ancient Codex                 20.4                   26.26    25.35              24.456667             24.078931            23.798899
5570  Backwards Ballcap                    Galactic Codex                 25.2                   27.74    27.50              26.253333             26.375416            26.392187
5569  Backwards Ballcap             Voltaic Hat Protector                 32.3                   27.12    26.70              29.206667             24.908493            28.383790
5542  Backwards Ballcap               It's a puzzle to me                 42.9                   29.38    28.2

In [69]:
unusuals_final_predictions_df[unusuals_final_predictions_df.columns.drop(["index","Predicted Price KNN","Predicted Price Forest",
                                                                          "Bagging","Learn Weights = False","Learn Weights = True"])].rename(
                                                                            columns = {"Price Pred. Average" : "Price"}).to_csv("Predictions.csv")

In [70]:
unusuals_df

,Name,Effect,Price,Class,Item Type,Equip Region,Grade,Unboxed From,Robo,Generation,Effect Community Rating,Crate Cosmetic,Months Since Last Price Update,Effect Median Price,Hat Median Price,Hat Rating,Effect Rating,Median Effect Rating,Median Hat Rating
0,Dread Knot,Burning Flames,30.0,Heavy,Cosmetic,Regular,No Grade,Old Series,False,First Gen,4.59375,True,9,104.00,14.0,-7.40,1.6,6.7375,-1.6000
1,Dread Knot,Circling Heart,NaN,Heavy,Cosmetic,Regular,No Grade,Old Series,False,First Gen,3.06250,True,-1,36.25,14.0,NaN,NaN,0.4500,-1.6000
2,Dread Knot,Circling Peace Sign,10.0,Heavy,Cosmetic,Regular,No Grade,Old Series,False,First Gen,2.21875,True,21,22.00,14.0,-1.20,-0.4,-0.8500,-1.6000
3,Dread Knot,Circling TF Logo,9.0,Heavy,Cosmetic,Regular,No Grade,Old Series,False,First Gen,2.34375,True,44,21.50,14.0,-1.25,-0.5,-0.9125,-1.6000
4,Dread Knot,Green Confetti,9.0,Heavy,Cosmetic,Regular,No Grade,Old Series,False,First Gen,1.40625,True,15,17.00,14.0,-0.80,-0.5,-1.3750,-1.6000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41276,Snowcapped,Magnetic Hat Protector,NaN,Medic,Cosmetic,Regular,Mercenary,Winter 2018 Cosmetic,False,Invasion,2.15625,False,-1,20.50,22.0,NaN,NaN,-0.9000,-0.1625
41277,Snowcapped,Voltaic Hat Protector,NaN,Medic,Cosmetic,Regular,Mercenary,Winter 2018 Cosmetic,False,Invasion,2.09375,False,-1,52.50,22.0,NaN,NaN,-1.1750,-0.1625
41278,Snowcapped,Galactic Codex,NaN,Medic,Cosmetic,Regular,Mercenary,Winter 2018 Cosmetic,False,Invasion,1.43750,False,-1,36.75,22.0,NaN,NaN,-1.1000,-0.1625
41279,Snowcapped,Ancient Codex,NaN,Medic,Cosmetic,Regular,Mercenary,Winter 2018 Cosmetic,False,Invasion,1.46875,False,-1,19.50,22.0,NaN,NaN,-1.2500,-0.1625
